## Data Introduction

In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [27]:
jobs = pd.read_csv("../data/fullsample.csv")
jobs["END"] = jobs["END"].str.replace("T", " ")
jobs.head(5)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0


The fullsample dataset contains job records, with one row per job.

Each job gets a unique ID, contained in the **JOBID** column.

Some jobs can be submitted as arrays of similar jobs. These are listed with an underscore in the JOBID, where the number after the underscore indicates the tasknumber. For example. JOBID 31781951 was an array job with 10 parts. 

In [5]:
jobs[jobs['JOBID'].str.contains('31781951')]

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
533,31781951_1,COMPLETED,2021-08-30T12:51:30,2021-09-08 02:17:41,16384Mn,10234.37M,12-00:00:00,8-13:26:11,1,12,production,0:0
534,31781951_2,COMPLETED,2021-08-30T12:51:30,2021-09-07 18:04:48,16384Mn,10247.40M,12-00:00:00,8-05:13:18,1,12,production,0:0
535,31781951_3,COMPLETED,2021-08-31T09:14:29,2021-09-08 16:36:06,16384Mn,10064.47M,12-00:00:00,8-07:21:37,1,12,production,0:0
536,31781951_4,COMPLETED,2021-09-01T01:59:50,2021-09-08 08:48:28,16384Mn,10004.80M,12-00:00:00,7-06:48:38,1,12,production,0:0
537,31781951_5,COMPLETED,2021-09-02T00:09:27,2021-09-08 23:58:57,16384Mn,9858.72M,12-00:00:00,6-23:49:30,1,12,production,0:0
538,31781951_6,COMPLETED,2021-09-02T16:19:55,2021-09-10 11:16:57,16384Mn,10065.06M,12-00:00:00,7-18:57:02,1,12,production,0:0
539,31781951_7,COMPLETED,2021-09-02T22:26:08,2021-09-10 18:48:31,16384Mn,10092.55M,12-00:00:00,7-20:22:23,1,12,production,0:0
540,31781951_8,COMPLETED,2021-09-03T10:54:14,2021-09-11 09:32:28,16384Mn,10146.98M,12-00:00:00,7-22:38:14,1,12,production,0:0
541,31781951_9,COMPLETED,2021-09-04T22:54:03,2021-09-12 16:16:04,16384Mn,10050.81M,12-00:00:00,7-17:22:01,1,12,production,0:0
542,31781951_10,COMPLETED,2021-09-06T06:54:35,2021-09-14 13:02:37,16384Mn,10042.53M,12-00:00:00,8-06:08:02,1,12,production,0:0


Jobs can have a few differents states, with the most common one being 'COMPLETED'. 

In [29]:
jobs["date"] = pd.to_datetime(jobs["END"], errors="coerce")
jobs1 = jobs.dropna(subset=["date"])
jobs1["date_hour"] = jobs1["date"].dt.strftime('%Y-%m-%d %H:%M:%S')
print(jobs1)


C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\3634595585.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  jobs["date"] = pd.to_datetime(jobs["END"], errors="coerce")


              JOBID      STATE                BEGIN                  END  \
1          30853133  COMPLETED  2021-08-06T11:36:09  2021-09-05 11:36:32   
2          30858137  COMPLETED  2021-08-06T19:04:39  2021-09-05 19:04:53   
3          30935078  COMPLETED  2021-08-09T16:52:51  2021-09-07 20:52:55   
4        31364111_2  COMPLETED  2021-08-17T07:45:07  2021-09-10 16:45:24   
5        31364111_3  COMPLETED  2021-08-17T07:45:07  2021-09-06 16:17:34   
...             ...        ...                  ...                  ...   
7395880    25493434  COMPLETED  2020-10-31T23:39:00  2020-10-31 23:40:46   
7395881    25493435  COMPLETED  2020-10-31T23:39:13  2020-10-31 23:40:38   
7395882    25493476  COMPLETED  2020-10-31T23:46:29  2020-10-31 23:49:43   
7395883    25493515  COMPLETED  2020-10-31T23:49:44  2020-10-31 23:51:40   
7395884    25493542  COMPLETED  2020-10-31T23:56:49  2020-10-31 23:59:57   

           REQMEM    USEDMEM      REQTIME     USEDTIME  NODES  CPUS  \
1        262144M

C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\3634595585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs1["date_hour"] = jobs1["date"].dt.strftime('%Y-%m-%d %H:%M:%S')


In [8]:
jobs['STATE'].value_counts()

STATE
COMPLETED              7375084
CANCELLED                 9055
FAILED                    3766
CANCELLED by 9201         1776
OUT_OF_MEMORY             1739
                        ...   
CANCELLED by 891323          1
CANCELLED by 889553          1
CANCELLED by 793827          1
CANCELLED by 790983          1
CANCELLED by 907426          1
Name: count, Length: 145, dtype: int64

The **BEGIN** field indicates when the job was started (initiated on a computer node).

The **END** field indicates when the job ended (completed, failed, or was cancelled while running).

The **REQMEM** field is the amount of memory requested in megabytes. It can be per-core/CPU (Mc) or per-node (Mn).



In [10]:
# Jobs where memory was requested per core.
jobs[jobs['REQMEM'].str[-2:] == 'Mc'].head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,date
501,31776583_1,COMPLETED,2021-08-30T10:16:59,2021-09-01 02:04:11,4096Mc,1792.43M,14-00:00:00,1-15:47:12,1,1,production,0:0,2021-09-01 02:04:11
502,31776584_12,COMPLETED,2021-08-30T10:17:00,2021-09-01 00:20:15,4096Mc,1792.43M,14-00:00:00,1-14:03:15,1,1,production,0:0,2021-09-01 00:20:15
915,31793401_958,COMPLETED,2021-08-31T19:36:46,2021-09-01 00:37:11,4096Mc,2788.05M,05:00:00,05:00:25,1,1,production,0:0,2021-09-01 00:37:11
916,31793401_987,COMPLETED,2021-08-31T20:33:46,2021-09-01 00:02:57,4096Mc,2779.27M,05:00:00,03:29:11,1,1,production,0:0,2021-09-01 00:02:57
4727,31813223_1296,COMPLETED,2021-08-31T19:42:46,2021-09-01 00:43:15,4096Mc,2786.44M,05:00:00,05:00:29,1,1,production,0:0,2021-09-01 00:43:15


In [11]:
# Jobs where memory was requested per node.
jobs[jobs['REQMEM'].str[-2:] == 'Mn'].head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,date
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0,NaT
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05 11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0,2021-09-05 11:36:32
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05 19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0,2021-09-05 19:04:53
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07 20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0,2021-09-07 20:52:55
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10 16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0,2021-09-10 16:45:24


The USEDMEM column is the amount of memory used in MB per node.

The requested time (REQTIME) and used time (USEDTIME) columns are in d-hh:mm:ss or hh:mm:ss for jobs less than one day in duration.

**NODES** is the number of servers used for the job. Most jobs are single node. For multiple node jobs, memory usage is the maximum over all nodes.

**CPUS** is the total number of CPU cores allocated to the job, and for multi-node jobs, this includes all nodes.

Most jobs are run in the "production" or "nogpfs" partition. The "debug" and "sam" partitions are test jobs that are expected to be short, and the "maxwell", "pascal", and "turing" partitions are for GPU resources.

In [13]:
jobs['PARTITION'].value_counts()

PARTITION
production              7019578
nogpfs                   147229
pascal                   124453
sam                       64967
turing                    21424
maxwell                   11278
cgw-maizie                 4309
debug                      1616
cgw-platypus                379
cgw-dsi-gw                  228
cgw-capra1                  157
cgw-dougherty1              125
cgw-horus                    61
cgw-cqs1                     28
cgw-hanuman                  21
cgw-sideshowbob              14
cgw-vm-qa-flatearth1          9
cgw-tbi01                     8
cgw-rocksteady                1
Name: count, dtype: int64

The **EXITCODE** gives the [exit code](https://www.agileconnection.com/article/overview-linux-exit-codes) for the job, with "0:0" indicating a successful job. Exit codes have two numbers, where if the first number is non-zero, it indicates a problem on the server side and if the second is nonzero, it indicates a problem on the user side.

In [15]:
jobs['EXITCODE'].value_counts()

EXITCODE
0:0      7384480
1:0         4958
0:15        1887
0:125       1739
0:9         1361
2:0          508
0:7          389
121:0         89
127:0         88
13:0          68
24:0          67
0:11          35
38:0          32
28:0          29
6:0           27
126:0         24
0:6           18
0:2           16
7:0           12
29:0          12
16:0           9
59:0           8
9:0            4
0:40           4
8:0            3
125:0          3
0:105          2
76:0           1
85:0           1
0:12           1
30:0           1
0:98           1
43:0           1
3:0            1
67:0           1
4:0            1
11:0           1
0:8            1
103:0          1
116:0          1
Name: count, dtype: int64

The slurm_wrapper_ce5.log and slurm_wrapper_ce6.log files contain logs of jobs submitted from the Open Science Grid.

In [36]:
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python')

ce5[3] =ce5[3].str.replace("time", "").astype(float)


                                  0          1        2         3  \
4770888  2021-10-07 21:58:06.738329  user 9203  retry 0  0.026772   
4770889  2021-10-07 21:58:15.931559  user 9201  retry 0  0.041662   
4770890  2021-10-07 21:58:48.900136  user 9221  retry 0  0.143490   
4770891  2021-10-07 21:59:11.314056  user 9203  retry 0  0.026599   
4770892  2021-10-07 21:59:17.693458  user 9201  retry 0  0.041465   

                    4                                                  5  
4770888  returncode 0  command ['/usr/bin/squeue', '-o', '%i %T', '-u...  
4770889  returncode 0  command ['/usr/bin/squeue', '-o', '%i %T', '-u...  
4770890  returncode 0  command ['/usr/bin/squeue', '-o', '%i %T', '-u...  
4770891  returncode 0  command ['/usr/bin/squeue', '-o', '%i %T', '-u...  
4770892  returncode 0  command ['/usr/bin/squeue', '-o', '%i %T', '-u...  


In [108]:
ce5["command"] = ce5[5].str.extract(r'([^/]+)$')
ce5["extract"] = ce5["command"].str.extract(r"(\w+)'")
ce5[3] = pd.to_numeric(ce5[3], errors="coerce")
ce5[0] = pd.to_datetime(ce5[0], errors="coerce")
ce5["Date"] = pd.to_datetime(ce5[0], errors="coerce")
ce5_drop = ce5.dropna(subset=["Date"])
ce5_filtered = ce5_drop[ce5_drop.loc[:,4] != "returncode 0"]
ce5_user = ce5_filtered[ce5_filtered.loc[:,1] == "user 9204"]
ce5_time = ce5_user[ce5_user[3] > 0.15]
filtered = ce5_time[(ce5_time[0] >= "2020-10-16") & (ce5_time[0] <= "2021-10-07")]
filtered2 = filtered[filtered["extract"] == "squeue"]

In [116]:
start_end_pairs = []

for i in range(len(filtered2) - 1):
    start_time = filtered2['Date'].iloc[i]
    end_time = filtered2['Date'].iloc[i + 1] 
    start_end_pairs.append((start_time, end_time))

# Convert to a DataFrame for better visualization
start_end_df = pd.DataFrame(start_end_pairs, columns=['start', 'end'])

results_list = []
for _, row in start_end_df.iterrows():
    start_time = row['start']
    end_time = row['end']
    
    # Filter jobs based on each datetime range
    jobs_filtered = jobs[(jobs["date"] >= start_time) & (jobs["date"] < end_time)]
    
    # Filter jobs where state is "COMPLETED"
    jobs2 = jobs_filtered[(jobs_filtered["STATE"] == "COMPLETED")]
    
    # Set the date as the index
    jobs2.set_index("date", inplace=True)
    
    # Resample by minute and count the number of jobs
    rolling_window = jobs2.resample('T').count()
    
    # Calculate job per minute and total number of jobs
    job_per_min = rolling_window["JOBID"].mean()
    sum_job = rolling_window["JOBID"].sum()
    
    # Print results for each datetime range
    results_list.append({"start": start_time, "end": end_time, "jobs_per_minute":job_per_min,"total_jobs": sum_job})
result_df = pd.DataFrame(results_list)
result_df['start'] = pd.to_datetime(result_df['start'])
result_df['end'] = pd.to_datetime(result_df['end'])
result_df["duration"] = result_df['end'] - result_df['start']
zero_duration = result_df[result_df["total_jobs"]==4].groupby("total_jobs")["duration"].mean()
print(zero_duration)
total_job_count = result_df["total_jobs"].value_counts(ascending=True)
print(total_job_count)



C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\3920315781.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\3920315781.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\3920315781.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\3920315781.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\3920315781.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version

total_jobs
4   0 days 00:00:00.973462
Name: duration, dtype: timedelta64[ns]
total_jobs
4         1
9         1
413       1
651       1
5829      1
       ... 
126       2
266       2
6         3
1         4
0       128
Name: count, Length: 157, dtype: int64


In [124]:
start_end_pairs = []

for i in range(len(filtered2) - 1):
    start_time = filtered2['Date'].iloc[i]
    end_time = filtered2['Date'].iloc[i + 1] 
    start_end_pairs.append((start_time, end_time))

# Convert to a DataFrame for better visualization
start_end_df = pd.DataFrame(start_end_pairs, columns=['start', 'end'])

results_list = []
for _, row in start_end_df.iterrows():
    start_time = row['start']
    end_time = row['end']
    
    # Filter jobs based on each datetime range
    jobs_filtered = jobs[(jobs["date"] >= start_time) & (jobs["date"] < end_time)]
    
    # Filter jobs where state is "COMPLETED"
    jobs2 = jobs_filtered[(jobs_filtered["STATE"] == "COMPLETED")]
    
    # Set the date as the index
    jobs2.set_index("date", inplace=True)
    
    # Resample by minute and count the number of jobs
    rolling_window = jobs2.resample('T').count()
    
    # Calculate job per minute and total number of jobs
    job_per_min = rolling_window["JOBID"].mean()
    sum_job = rolling_window["JOBID"].sum()
    
    # Print results for each datetime range
    results_list.append({"start": start_time, "end": end_time, "jobs_per_minute":job_per_min,"total_jobs": sum_job})
result_df = pd.DataFrame(results_list)
result_df['start'] = pd.to_datetime(result_df['start'])
result_df['end'] = pd.to_datetime(result_df['end'])
result_df["duration"] = result_df['end'] - result_df['start']
zero_duration = result_df[result_df["total_jobs"]==1].groupby("total_jobs")["duration"].mean()
print(zero_duration)
total_job_count = result_df["total_jobs"].value_counts(ascending=True)
one_job_selected = result_df[result_df["total_jobs"] == 1]
print(one_job_selected)



C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\1460723363.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\1460723363.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\1460723363.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\1460723363.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  rolling_window = jobs2.resample('T').count()
C:\Users\vetdd\AppData\Local\Temp\ipykernel_33660\1460723363.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version

total_jobs
1   0 days 00:00:19.040923
Name: duration, dtype: timedelta64[ns]
                         start                        end  jobs_per_minute  \
40  2021-09-01 10:10:55.865848 2021-09-01 10:10:56.731863              1.0   
61  2021-09-08 16:29:28.918036 2021-09-08 16:29:53.478130              1.0   
268 2021-09-24 06:17:13.198701 2021-09-24 06:17:33.307387              1.0   
285 2021-09-24 12:45:07.724629 2021-09-24 12:45:38.353526              1.0   

     total_jobs               duration  
40            1 0 days 00:00:00.866015  
61            1 0 days 00:00:24.560094  
268           1 0 days 00:00:20.108686  
285           1 0 days 00:00:30.628897  


For this project, we are interested in jobs from user 9204 (the test user) where the command starts with '/usr/bin/squeue', the returncode is non-zero and the time is greater than 15. These conditions indicate that the scheduler becaem unresponsive at that point in time.